In [1]:
%pip install --upgrade tdmclient

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import tdmclient Notebook environment:
import tdmclient.notebook
await tdmclient.notebook.start()

Run les 2 cellules suivantes pour vérifier que le robot roule droit et checker qu'il est pas bourré comme le mien...

In [3]:
motor_left_target = 50
motor_right_target = 50

In [4]:
motor_left_target = 0
motor_right_target = 0

Run ce programme pour faire la local obstacle avoidance

In [5]:
%%run_python --wait


speed0 = 50       # nominal speed
obstThrL = 10      # low obstacle threshold to switch state 1->0
obstThrH = 20      # high obstacle threshold to switch state 0->1
obstSpeedGain = 5  # /100 (actual gain: 5/100=0.05)

state = 0          # 0= go to goal, 1=obstacle avoidance
obst = [0,0]            # measurements from left and right prox sensors

timer_period[0] = 10   # 10ms sampling time

proxsens1 = 0   #proximity sensor 1
proxsens5 = 4   #proximity sensor 2
go2goal = 0
obstavoid = 1

@onevent 
def timer0():
    global prox_horizontal, motor_left_target, motor_right_target, state, obst, obstThrH, obstThrL, obstSpeedGain, speed0, speedGain 
    # acquisition from the proximity sensors to detect obstacles
    obst = [prox_horizontal[proxsens1], prox_horizontal[proxsens5]]

    # tdmclient does not support yet multiple and/or in if statements:
    if state == go2goal: 
        # switch from goal tracking to obst avoidance if obstacle detected
        if (obst[0] > obstThrH) or (obst[1] > obstThrH):
            state = 1
            
    elif state == obstavoid:
        if (obst[0] < obstThrL) and (obst[1] < obstThrL):
            # switch from obst avoidance to goal tracking if obstacle got unseen
            state = 0
                
    if  state == obstavoid:
        leds_top = [30,30,30]
        # obstacle avoidance: accelerate wheel near obstacle
        motor_left_target = speed0 + obstSpeedGain * (obst[0] // 100)
        motor_right_target = speed0 + obstSpeedGain * (obst[1] // 100)
        
    elif state == go2goal:
        # goal tracking: turn toward the goal
        leds_top = [0,0,0]
        motor_left_target = 100
        motor_right_target = 100

        

Exception: Infinite loops not allowed